In [0]:
# Use this code if using Colab to run Selenium

# Make sure to go to Runtime -> Change runtime type and select Python 3 as runtime type and GPU as hardware accelerator

# !kill -9 -1 # Use this line to delete this VM and start a new one. 
# The above line deletes all files and folders from the current VM and allocates a new one.
!apt-get update
!pip install beautifulsoup4
!pip install selenium
# !apt-get -q update # to update ubuntu to correctly run apt install
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup, Tag, NavigableString
from selenium.webdriver.chrome.options import Options
import csv
import random
import time
import pandas as pd

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [5,944 B]

In [0]:
class SeleniumCraigslistScraper(object):
    pass

    def __init__(self, number_reviews_scrape):
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome(options=chrome_options)
        self.delay = 5
        self.reviews = []
        self.beers = []
        self.number_reviews_scrape = number_reviews_scrape

    def go_to_page(self, url, element_type=None, element_value=None):
        try:
            self.driver.get(url)
            if element_type is not None or element_value is not None:
              if element_type == 'id':
                  self.wait(element_id=element_value)
              elif element_type == 'name':
                  self.wait(element_name=element_value)
        except TimeoutException:
            print('Loading took too long')
        except NoSuchElementException:
            print(f"Couldn\'t find {element_value} D:")
        except Exception as e:
            print(str(e))


    def wait(self, element_id=None, element_class_name=None, element_name=None):
        delay = random.randint(1, 6)
        wait = WebDriverWait(self.driver, delay)
        if element_id is not None:
            wait.until(EC.presence_of_element_located((By.ID, element_id)))
        if element_class_name is not None:
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, element_class_name)))
        if element_name is not None:
            wait.until(EC.presence_of_element_located((By.NAME, element_name)))
        else:
            time.sleep(delay)

    def get_list_of_beers(self):
        beers = self.driver.find_element_by_id('ba-content').find_element_by_tag_name('table').find_elements_by_tag_name('tr')
        self.beers = beers[1:len(beers)]  # removes the first <tr> (doesn't contain beer)

    def iterate_through_beers(self):
        init_count = len(self.beers)
        for i in range(1, init_count):
            if len(self.reviews) < self.number_reviews_scrape:
                # get page source to prevent stale elements error
                beers = self.driver.find_element_by_id('ba-content').find_element_by_tag_name('table').find_elements_by_tag_name('tr')
                beer = beers[i]
                beer.find_elements_by_tag_name('td')[1].find_element_by_tag_name('a').click()
                self.scrape_page_reviews()
                self.driver.execute_script("window.history.go(-1)")
                self.wait()

    def scrape_page_reviews(self):
        html = self.driver.page_source
        soup = BeautifulSoup(html, "lxml")
        product_name = soup.find('div', {'class': 'titleBar'})#.h1.text
        for span in product_name.findAll('span'):
            span.decompose()
        product_name = soup.find('div', {'class': 'titleBar'}).h1.text
        for user_review in soup.find_all('div', {'id': 'rating_fullview_content_2'}):
            user_score = user_review.find('span', {'class': 'BAscore_norm'}).text
            out_of_score = user_review.find('span', {'class': 'rAvg_norm'}).text.split('/')[1]

            for thing in user_review.find_all(['br', 'i', 'span', 'div']):
                thing.decompose()

            review_text = user_review.text
            if 'rDev' in review_text[0:10]:
                review_text = review_text[6:]
                
            review_text = review_text.strip()
            
            if '=' in review_text[0:1]:
                review_text = review_text[1:]
                
            review_text = self.deEmojify(review_text)
            
            self.reviews.append({'product name': product_name, 'product review': review_text, 'user rating': user_score})
            if len(self.reviews) % 100 == 0:
              print(f'have collected {len(self.reviews)} reviews')

    def deEmojify(self, inputString):
        return inputString.encode('ascii', 'ignore').decode('ascii')

    def create_pandas_csv(self, filename='beerviews.csv'):
        df = pd.DataFrame(self.reviews)
        df.to_csv(filename, encoding='utf-8', index=False)

    def quit_driver(self):
        self.driver.quit()

In [0]:
url = 'https://www.beeradvocate.com/beer/top-rated/'
number_reviews_scrape = 5000
try:  
  scraper = SeleniumCraigslistScraper(number_reviews_scrape=number_reviews_scrape)
  scraper.go_to_page(url, 'id', 'ba-content')
  scraper.get_list_of_beers()
  scraper.iterate_through_beers()
  scraper.quit_driver()
  scraper.create_pandas_csv('beerreviews.csv')
except Exception as e:
  print(str(e))
  scraper.quit_driver()

have collected 100 reviews
have collected 200 reviews
have collected 300 reviews
have collected 400 reviews
have collected 500 reviews
have collected 600 reviews
have collected 700 reviews
have collected 800 reviews
have collected 900 reviews
have collected 1000 reviews
have collected 1100 reviews
have collected 1200 reviews
have collected 1300 reviews
have collected 1400 reviews
have collected 1500 reviews
have collected 1600 reviews
have collected 1700 reviews
have collected 1800 reviews
have collected 1900 reviews
have collected 2000 reviews
have collected 2100 reviews
have collected 2200 reviews
have collected 2300 reviews
have collected 2400 reviews
have collected 2500 reviews
have collected 2600 reviews
have collected 2700 reviews
have collected 2800 reviews
have collected 2900 reviews
have collected 3000 reviews
have collected 3100 reviews
have collected 3200 reviews
have collected 3300 reviews
have collected 3400 reviews
have collected 3500 reviews
have collected 3600 reviews
h